# 11.3 항공 사진 내 선인장 식별 경진대회 베이스라인 모델
- [항공 사진 내 선인장 식별 경진대회 링크](https://www.kaggle.com/c/aerial-cactus-identification)
- [베이스라인 코드 참고 링크](https://www.kaggle.com/bonhart/simple-cnn-on-pytorch-for-beginers)

## 11.3.1 시드 값 고정 및 GPU 장비 설정

### 시드 값 고정

In [17]:
%config Completer.use_jedi = False

In [18]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

* GPU 장비 설정

In [19]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [20]:
device

device(type='cuda')

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

### 11.3.2 데이터 준비

In [22]:
import pandas as pd

data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [23]:
from zipfile import ZipFile

with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

* 훈련 데이터, 검증 데이터 분석

In [24]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(labels, test_size=0.1, stratify=labels['has_cactus'], random_state=50)

In [25]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

훈련 데이터 개수: 15750
검증 데이터 개수: 1750


* 데이터셋 클래스 정의

In [26]:
import cv2
from torch.utils.data import Dataset

In [27]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label

* 데이터셋 생성

In [28]:
from torchvision import transforms

transform = transforms.ToTensor()

In [29]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

* 데이터 로더 생성

In [30]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

### 11.3.3 모델 생성

In [31]:
import torch.nn as nn
import torch.nn.functional as F

In [32]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.fc(x)
        return x

In [33]:
model = Model().to(device)

In [34]:
model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

### 11.3.4 모델 훈련

In [35]:
criterion = nn.CrossEntropyLoss()

* 옵티마이저 설정

In [36]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [39]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    
    for images, labels in loader_train:
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item()
        loss.backward()
        # 가중치 계산
        optimizer.step()
        
    print(f'에폭 {epoch+1}/{epochs} - 손실값: {epoch_loss/len(loader_train):.4f}')

에폭 1/10 - 손실값: 0.5237
에폭 2/10 - 손실값: 0.3416
에폭 3/10 - 손실값: 0.2392
에폭 4/10 - 손실값: 0.2010
에폭 5/10 - 손실값: 0.1774
에폭 6/10 - 손실값: 0.1632
에폭 7/10 - 손실값: 0.1514
에폭 8/10 - 손실값: 0.1416
에폭 9/10 - 손실값: 0.1374
에폭 10/10 - 손실값: 0.1297


### 11.3.5 성능 검증

In [41]:
from sklearn.metrics import roc_auc_score

true_list = []
preds_list = []

In [42]:
model.eval()

with torch.no_grad():
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu()
        
        preds_list.extend(preds)
        true_list.extend(true)

print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list): .4f}')
        

검증 데이터 ROC AUC :  0.9901


### 11.3.6 예측 및 결과 제출

In [43]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

In [44]:
model.eval()

preds = []

with torch.no_grad():
    for images, _ in loader_test:
        images = images.to(device)
        
        outputs = model(images)
        pred_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        preds.extend(pred_part)

* 결과 제출

In [45]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [46]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')